In [1]:
import os
import sys
import string

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
import spacy
import re

In [4]:
!  ls ../data

HHCAHPS_MeasureDateRange_Oct2020.csv HH_National_Oct2020.csv
HHCAHPS_National_Oct2020.csv         HH_Provider_Oct2020.csv
HHCAHPS_Provider_Oct2020.csv         HH_State_Oct2020.csv
HHCAHPS_State_Oct2020.csv            HH_Zip_Oct2020.csv
HH_MeasureDateRange_Oct2020.csv


## Exploring the Data

In [5]:
data_state= pd.read_csv('../data/HHCAHPS_State_Oct2020.csv')
data_state[data_state['State']=='CA']

,State,Percent of patients who reported that their home health team gave care in a professional way,Percent of patients who reported that their home health team communicated well with them,"Percent of patients who reported that their home health team discussed medicines, pain, and home safety with them",Percent of patients who gave their home health agency a rating of 9 or 10 on a scale from 0 (lowest) to 10 (highest),"Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family",Number of completed Surveys,Response rate
4,CA,86,83,82,80,74,107153,20


In [6]:
data_survey1 = pd.read_csv('../data/HHCAHPS_Provider_Oct2020.csv')
data_survey1

,CMS Certification Number (CCN),HHCAHPS Survey Summary Star Rating,HHCAHPS Survey Summary Star Rating Footnote,Star Rating for health team gave care in a professional way,Footnote for Star Rating for gave care in a professional way,Percent of patients who reported that their home health team gave care in a professional way,Footnote for Percent of patients who reported that their home health team gave care in a professional way,Star Rating for health team communicated well with them,Footnote for Star Rating for communicated well with them,Percent of patients who reported that their home health team communicated well with them,...,Footnote for Star Rating for overall care from agency,Percent of patients who gave their home health agency a rating of 9 or 10 on a scale from 0 (lowest) to 10 (highest),Footnote for Percent of patients who gave their home health agency a rating of 9 or 10 on a scale from 0(lowest) to 10(highest),"Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family","Footnote for Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family",Number of completed Surveys,Footnote for number of completed surveys,Response rate,Footnote for response rate,Footnote Number
0,17000,5,NaN,4,NaN,91,NaN,5,NaN,88,...,NaN,88,NaN,81,NaN,794,NaN,25,NaN,NaN
1,17009,5,NaN,4,NaN,91,NaN,5,NaN,90,...,NaN,88,NaN,85,NaN,290,NaN,33,NaN,NaN
2,17013,5,NaN,5,NaN,93,NaN,5,NaN,93,...,NaN,90,NaN,89,NaN,261,NaN,36,NaN,NaN
3,17014,4,NaN,4,NaN,89,NaN,4,NaN,85,...,NaN,85,NaN,84,NaN,125,NaN,31,NaN,NaN
4,17016,5,NaN,5,NaN,92,NaN,5,NaN,90,...,NaN,91,NaN,85,NaN,204,NaN,30,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11171,747977,Not Available,12,Not Available,12,Not Available,12,Not Available,12,Not Available,...,12,Not Available,12,Not Available,12,Not Available,12,Not Available,12,12
11172,747978,Not Available,12,Not Available,12,Not Available,12,Not Available,12,Not Available,...,12,Not Available,12,Not Available,12,Not Available,12,Not Available,12,12
11173,747980,Not Available,12,Not Available,12,Not Available,12,Not Available,12,Not Available,...,12,Not Available,12,Not Available,12,Not Available,12,Not Available,12,12
11174,747983,Not Available,12,Not Available,12,Not Available,12,Not Available,12,Not Available,...,12,Not Available,12,Not Available,12,Not Available,12,Not Available,12,12


In [7]:
df_survey2 = pd.read_csv('../data/HHCAHPS_MeasureDateRange_Oct2020.csv')
df_survey2

,Measure Name,Measure Date Range
0,How often the home health team gave care in a ...,"January 1, 2019 - December 31, 2019"
1,How well did the home health team communicate ...,"January 1, 2019 - December 31, 2019"
2,"Did the home health team discuss medicines, pa...","January 1, 2019 - December 31, 2019"
3,How do patients rate the overall care from the...,"January 1, 2019 - December 31, 2019"
4,Would patients recommend the home health agenc...,"January 1, 2019 - December 31, 2019"


In [8]:
df_survey3 = pd.read_csv('../data/HHCAHPS_State_Oct2020.csv')
df_survey3[df_survey3['State']=='CA']

,State,Percent of patients who reported that their home health team gave care in a professional way,Percent of patients who reported that their home health team communicated well with them,"Percent of patients who reported that their home health team discussed medicines, pain, and home safety with them",Percent of patients who gave their home health agency a rating of 9 or 10 on a scale from 0 (lowest) to 10 (highest),"Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family",Number of completed Surveys,Response rate
4,CA,86,83,82,80,74,107153,20


In [9]:
df_survey4 = pd.read_csv('../data/HHCAHPS_National_Oct2020.csv')
df_survey4

,Country,Percent of patients who reported that their home health team gave care in a professional way,Percent of patients who reported that their home health team communicated well with them,"Percent of patients who reported that their home health team discussed medicines, pain, and home safety with them",Percent of patients who gave their home health agency a rating of 9 or 10 on a scale from 0 (lowest) to 10 (highest),"Percent of patients who reported YES, they would definitely recommend the home health agency to friends and family",Number of completed Surveys,Response rate
0,Nation,88,85,83,84,78,1144837,27


In [10]:
df = pd.read_csv('../data/HH_Provider_Oct2020.csv')

In [11]:
df_cal = df[df['State'] == 'CA']

In [12]:
df_cal.reset_index(drop=True, inplace=True)

In [13]:
df_cal.shape

(1585, 70)

In [14]:
df_cal.head(5)

,State,CMS Certification Number (CCN),Provider Name,Address,City,ZIP,Phone,Type of Ownership,Offers Nursing Care Services,Offers Physical Therapy Services,...,PPR Denominator,PPR Observed Rate,PPR Risk-Standardized Rate,PPR Risk-Standardized Rate (Lower Limit),PPR Risk-Standardized Rate (Upper Limit),PPR Performance Categorization,Footnote for PPR Risk-Standardized Rate,"How much Medicare spends on an episode of care at this agency, compared to Medicare spending across all agencies nationally","Footnote for how much Medicare spends on an episode of care at this agency, compared to Medicare spending across all agencies nationally","No. of episodes to calc how much Medicare spends per episode of care at agency, compared to spending at all agencies (national)"
0,CA,57002,"CENTRAL COAST VNA & HOSPICE, INC",5 LOWER RAGSDALE DRIVE,MONTEREY,93942,8313726668,VOLUNTARY NON-PROFIT - PRIVATE,Yes,Yes,...,3125.0,3.94,3.77,3.20,4.38,Same As National Rate,NaN,0.90,NaN,4373
1,CA,57003,SUTTER VISITING NURSE ASSOCIATION AND HOSPICE,"2800 LEAVENWORTH STREET, SUITE 350A",SAN FRANCISCO,94133,4157494200,VOLUNTARY NON-PROFIT - OTHER,Yes,Yes,...,1478.0,2.30,2.94,2.33,3.76,Same As National Rate,NaN,0.95,NaN,2410
2,CA,57004,SUTTER VISITING NURSE ASSOCIATION AND HOSPICE,"5099 COMMERCIAL CIRCLE, SUITE 205-A",CONCORD,94520,9256774240,VOLUNTARY NON-PROFIT - PRIVATE,Yes,Yes,...,3747.0,3.47,3.26,2.82,3.75,Same As National Rate,NaN,0.94,NaN,6027
3,CA,57007,SUTTER VISITING NURSE ASSOCIATION AND HOSPICE,"2880 SOQUEL AVENUE, SUITE 10",SANTA CRUZ,95062,8314796625,VOLUNTARY NON-PROFIT - PRIVATE,Yes,Yes,...,2462.0,1.79,2.73,2.18,3.40,Better Than National Rate,NaN,0.88,NaN,4662
4,CA,57008,VNA HOSPICE AND PALLIATIVE CARE OF SOUTHERN CA,1317 W FOOTHILL BLVD SUITE 235,UPLAND,91786,9096243574,VOLUNTARY NON-PROFIT - OTHER,Yes,Yes,...,177.0,6.21,3.86,2.86,5.38,Same As National Rate,NaN,0.91,NaN,420


In [15]:
missing_df_cal = df_cal.isnull().sum()
missing_df_cal

State                                                                                                                                          0
CMS Certification Number (CCN)                                                                                                                 0
Provider Name                                                                                                                                  0
Address                                                                                                                                        0
City                                                                                                                                           0
                                                                                                                                            ... 
PPR Performance Categorization                                                                                                    

In [16]:
for name,value in zip(missing_df_cal.index, missing_df_cal):
    print(f'{name} : {value}')

State : 0
CMS Certification Number (CCN) : 0
Provider Name : 0
Address : 0
City : 0
ZIP : 0
Phone : 0
Type of Ownership : 0
Offers Nursing Care Services : 0
Offers Physical Therapy Services : 0
Offers Occupational Therapy Services : 0
Offers Speech Pathology Services : 0
Offers Medical Social Services : 0
Offers Home Health Aide Services : 0
Date Certified : 0
Quality of patient care star rating : 378
Footnote for quality of patient care star rating : 1207
How often the home health team began their patients' care in a timely manner : 339
Footnote for how often the home health team began their patients' care in a timely manner : 1246
How often the home health team taught patients (or their family caregivers) about their drugs : 339
Footnote for how often the home health team taught patients (or their family caregivers) about their drugs : 1246
How often the home health team checked patients' risk of falling : 349
Footnote for how often the home health team checked patients' risk of fall

In [17]:
# Remove footnote columns
df_cal = df_cal.loc[:, ~df_cal.columns.str.startswith('Footnote')]

In [18]:
df_cal.shape

(1585, 49)

In [19]:
df_cal.columns

Index(['State', 'CMS Certification Number (CCN)', 'Provider Name', 'Address',
       'City', 'ZIP', 'Phone', 'Type of Ownership',
       'Offers Nursing Care Services', 'Offers Physical Therapy Services',
       'Offers Occupational Therapy Services',
       'Offers Speech Pathology Services', 'Offers Medical Social Services',
       'Offers Home Health Aide Services', 'Date Certified',
       'Quality of patient care star rating',
       'How often the home health team began their patients' care in a timely manner',
       'How often the home health team taught patients (or their family caregivers) about their drugs',
       'How often the home health team checked patients' risk of falling',
       'How often the home health team checked patients for depression',
       'How often the home health team determined whether patients received a flu shot for the current flu season',
       'How often the home health team made sure that their patients received a pneumococcal vaccine (pneumon

## PDF Reading via Optical Character Recognition (OCR)

In [ ]:
pdf_path = 'ex1.pdf'

In [ ]:
%%time
pages = convert_from_path(pdf_path)#, dpi=500)

In [ ]:
%%time
# Read pdf file as JPG images, save to disk
im_count = 0
for page in pages:
    fname = f'page_{str(im_count)}.jpg'
    page.save(f'images/{fname}', 'JPEG')
    im_count += 1

In [ ]:
filelimit = im_count-1

In [ ]:
%%time
# Read each image, convert to text via OCR, write to text file
output = 'output.txt'
with open(output, 'a') as f:
    for i in range(1, filelimit + 1):
        fname = f'images/page_{str(i)}.jpg'
        image = Image.open(fname)
        text = str(((pytesseract.image_to_string(image))))
        text = text.replace('-\n', '')    
        f.write(text)

### Read Text Data

In [36]:
output = 'output.txt'
with open(output, 'r')  as f:
    text = f.read()

In [37]:
text[4000:4500]

'\ninitiated. This patient will be followed by a physician (MONICA MUNOZ NADAL NP) who will\nbe assuming the care of the patient.\n\nPrinted by HO, RAINBOW D [C777696] at 5/18/2021 2:51:21 PM\n\x0cBrown, Stephen W (MRN 110004285604) DOB: 01/28/1945 Encounter Date: 05/17/2021\n\nMRN: 110004285604\n\nBrown, Stephen W\n\nED to Hosp-Admission Last attending: Pham, Y Long (M.D.)\n\n5/17/2021 - present (1 day) Principal problem: ACUTE PULMONARY EMBOLISM\nStatus: Current\n\nSSF-HOSPITAL\n\nDischarge Summary Pham, Y Long (M.'

### Clean-up Text

In [38]:
# Initalize ordinal translations
translation_ws = {ord(c): ' ' for c in string.whitespace}
translation_punc = {ord(c): '' for c in string.punctuation}

In [39]:
# Translate/ clean text
text = text.translate(translation_ws)
text = text.translate(translation_punc)
text = ' '.join(text.split())

In [40]:
text[4000:4500]

' EMBOLISM Status Current SSFHOSPITAL Discharge Summary Pham Y Long MD Physician » Hospital Mecicine « 5182021 1134 AM W Date of Admission 5172021 Date of Discharge 5182021 Issues to be Addressed at Followup Switch to pradaxa 150mg bid or xarelto 20mg daily on 6821 after 21 days treatment with xarelto 15mg bid Pending Study Results at Discharge Unresulted Lab Orders No orders found from 4182021 to 5192021 Final Diagnoses Active Hospital Problems i cn TT a gnosis Principal ACUTE PULMONARY EMBOLISM'

### Extract Zipcode
California zipcodes start with a "9"

In [41]:
pattern = re.compile(r'.*(9\d{4})')

In [42]:
re.findall(pattern, text)

['94122']

In [43]:
zipcode = '94122'

### Remove other Digits

In [44]:
translation_dig = {ord(c): '' for c in string.digits}
text = text.translate(translation_dig)

In [45]:
text[4000:4500]

'oses Active Hospital Problems i cn TT a gnosis Principal ACUTE PULMONARY EMBOLISM PLEURITIC CHEST PAIN ACUTE HYPOXEMIC RESPIRATORY FAILURE CELLULITIS LEUKOCYTOSIS Resolved Hospital Problems No resolved problems to display Discharge Disposition Assisted livign Medications instructions follow up information See patient Discharge InstructionsAVS Hospital Course and Significant Findings  yo male ho htn DM COPD TIA on plavix OSA dementia mostly bedbound who resides at AJ boarding home and on palliati'

### Remove Stop Words & Get List of Words

In [46]:
def text_process(s):
    nlp = spacy.load('en_core_web_sm')
    string = nlp(s)
    filtered_list = []
    for token in string:
        if token.is_stop or len(token) < 3:
            continue
        filtered_list.append(token.text.lower())
#     filtered_string = ' '.join(filtered_list)
    return filtered_list

In [47]:
word_list = text_process(text)

In [48]:
print(word_list[3500:3550])

['needed', 'pain', 'fever', 'exceed', 'tablets', 'hours', 'clopidogrel', 'plavix', 'oral', 'tablet', 'mouth', 'tab', 'daily', 'tamsulosin', 'flomax', 'capsule', 'oral', 'cap', 'mouth', 'daily', 'bedtime', 'prostate', 'urination', 'humulin', 'nph', 'insulin', 'inject', 'units', 'unitml', 'subq', 'susp', 'subcutaneously', 'times', 'day', 'directed', 'diclofenac', 'sodium', 'voltaren', 'apply', 'grams', 'gel', 'knees', 'times', 'day', 'needed', 'pain', 'bisacodyl', 'dulcolax', 'unwrap', 'rect']


In [49]:
len(word_list)

9628

## Recommendation
- A naive recommendation as a first prototype is a keyword match from our list of words to the topic questions in the data

**Core Questions for Recommendation:**
- How often the home health team began their patients' care in a timely manner
- How often the home health team taught patients (or their family caregivers) about their drugs
- How often the home health team checked patients' risk of falling
- How often the home health team checked patients for depression
- How often the home health team determined whether patients received a flu shot for the current flu season
- How often the home health team made sure that their patients received a pneumococcal vaccine (pneumonia shot)
- With diabetes, how often the home health team got doctor's orders, gave foot care, and taught patients about foot care
- How often patients got better at walking or moving around
- How often patients got better at getting in and out of bed
- How often patients got better at bathing
- How often patients' breathing improved
- How often patients' wounds improved or healed after an operation
- How often patients got better at taking their drugs correctly by mouth
- How often home health patients had to be admitted to the hospital
- How often patients receiving home health care needed urgent, unplanned care in the ER without being admitted
- Changes in skin integrity post-acute care: pressure ulcer/injury
- How often physician-recommended actions to address medication issues were completely timely

In [50]:
# Rename columns with Q1, Q2, etc.
long_questions = df_cal.columns[16:33]
short_questions = [f'Q{i+1}' for i in range(17)]

new_columns = []
for i,col in enumerate(df_cal.columns):
    if col not in long_questions:
        new_columns.append(col)
    else:
        new_columns.append(short_questions[i-16])

In [51]:
df_cal.columns = new_columns

**Potential Keywords for each Question:**
- Q1: How often the home health team began their patients' care in a timely manner
    - **General question rated without keywords**
- Q2: How often the home health team taught patients (or their family caregivers) about their drugs
    - **General question rated without keywords**
- Q3: How often the home health team checked patients' risk of falling
    - "falling", "fall", "fell", "tripped", "trip", "tumble"
- Q4: How often the home health team checked patients for depression
    - "depression", "mental", "bipolar", "sad", "upset", "trauma", "traumatic"
- Q5: How often the home health team determined whether patients received a flu shot for the current flu season
    - "flu"
- Q6: How often the home health team made sure that their patients received a pneumococcal vaccine (pneumonia shot)
    - "pneumonia", "pneumococcal"
- Q7: With diabetes, how often the home health team got doctor's orders, gave foot care, and taught patients about foot care
    - "diabetes", "foot"
- Q8: How often patients got better at walking or moving around
    - "walk", "walking", "moving"
- Q9: How often patients got better at getting in and out of bed
    - "bed"
- Q10: How often patients got better at bathing
    - "bathing", "bathe", "bath"
- Q11: How often patients' breathing improved
    - "breathing", "breathe", "oxygen"
- Q12: How often patients' wounds improved or healed after an operation
    - "wound", "wounds", "cut", "slice", "lesion", "gash", "puncture", "slash", "laceration", "tear", "rent", "puncture", "slash", "sore", "graze", "scratch", "scrape", "abrasion", "bruise", "contusion", "trauma"
- Q13: How often patients got better at taking their drugs correctly by mouth
    - **General question rated without keywords**
- Q14: How often home health patients had to be admitted to the hospital
    - **General question rated without keywords**
- Q15: How often patients receiving home health care needed urgent, unplanned care in the ER without being admitted
    - **General question rated without keywords**
- Q16: Changes in skin integrity post-acute care: pressure ulcer/injury
    - "ulcer", "pressure", "skin", "post-acute"
- Q17: How often physician-recommended actions to address medication issues were completely timely
    - **General question rated without keywords**

In [52]:
# Create keyword lists
q3_keys = ["falling", "fall", "fell", "tripped", "trip", "tumble"]
q4_keys = ["depression", "mental", "bipolar", "sad", "upset", "trauma", "traumatic"]
q5_keys = ["flu"]
q6_keys = ["pneumonia", "pneumococcal"]
q7_keys = ["diabetes", "foot"]
q8_keys = ["walk", "walking", "moving"]
q9_keys = ["bed"]
q10_keys = ["bathing", "bathe", "bath"]
q11_keys = ["breathing", "breathe", "oxygen"]
q12_keys = ["wound", "wounds", "cut", "slice", "lesion",\
            "gash", "puncture", "slash", "laceration", "tear",\
            "rent", "puncture", "slash", "sore", "graze", "scratch",\
            "scrape", "abrasion", "bruise", "contusion", "trauma"]
q16_keys = ["ulcer", "pressure", "skin", "post-acute"]

In [53]:
# Create "flagged" dictionary if question topic needed for patient
flagged_qtopic = {f'Q{i}':False for i in range(3,13)}
flagged_qtopic['Q16'] = False

In [54]:
# Check all words in chart, if word is a keyword, flag that question
for word in word_list:
    if word in q3_keys:
        flagged_qtopic['Q3'] = True
    if word in q4_keys:
        flagged_qtopic['Q4'] = True
    if word in q5_keys:
        flagged_qtopic['Q5'] = True
    if word in q6_keys:
        flagged_qtopic['Q6'] = True
    if word in q7_keys:
        flagged_qtopic['Q7'] = True
    if word in q8_keys:
        flagged_qtopic['Q8'] = True
    if word in q9_keys:
        flagged_qtopic['Q9'] = True
    if word in q10_keys:
        flagged_qtopic['Q10'] = True
    if word in q11_keys:
        flagged_qtopic['Q11'] = True
    if word in q12_keys:
        flagged_qtopic['Q12'] = True
    if word in q16_keys:
        flagged_qtopic['Q16'] = True

In [55]:
flagged_qtopic

{'Q3': True,
 'Q4': True,
 'Q5': False,
 'Q6': False,
 'Q7': True,
 'Q8': False,
 'Q9': True,
 'Q10': False,
 'Q11': True,
 'Q12': True,
 'Q16': True}

So in this case we "don't care" about agencies that do the flu shot well, pnuemonia shot well, bed, or bathing, but really care about all other specific services. We will compare those only:

### TODO: Filter by Zipcode
- Since home health services comes to you, they have their own coverage of which zipcodes they can service